# Noise Contrastive Estimation to Calculate Absolute Free Energies

In [107]:
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt

import numpy as np
import torch
import pickle
import scipy.integrate as integrate
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
torch.set_default_dtype(torch.double)

### Define Helper Functions

In [108]:
def compute_Muller_potential(beta, x): #x must be type tensor
    A = (-200., -100., -170., 15.)
    b = (0., 0., 11., 0.6)    
    ac = (x.new_tensor([-1.0, -10.0]),
          x.new_tensor([-1.0, -10.0]),
          x.new_tensor([-6.5, -6.5]),
          x.new_tensor([0.7, 0.7]))
    
    x0 = (x.new_tensor([ 1.0, 0.0]),
          x.new_tensor([ 0.0, 0.5]),
          x.new_tensor([-0.5, 1.5]),
          x.new_tensor([-1.0, 1.0]))
    
    U = 0    
    for i in range(4):
        diff = x - x0[i]
        U = U + A[i]*torch.exp(torch.sum(ac[i]*diff**2, -1) + b[i]*torch.prod(diff, -1))

    return beta*U

In [109]:
def compute_Muller_potential_point_nobeta(r):
    """
    Computes the Muller potential at a point r = (x, y). r does not have to be a tensor
    """
    x = r[0]
    y = r[1]
    A = (-200., -100., -170., 15.)
    a = (-1, -1, -6.5, 0.7)
    b = (0., 0., 11., 0.6) 
    c = (-10, -10, -6.5, 0.7)
    x0 = (1, 0, -0.5, -1)
    y0 = (0, 0.5, 1.5, 1)

    result = 0
    for k in range(4):
        result += A[k]*np.exp(a[k]*(x-x0[k])**2 + b[k]*(x-x0[k])*(y-y0[k])+ c[k]*(y-y0[k])**2)
    return result 

### Define Model

In [110]:
class NCE(nn.Module):
    def __init__(self):
        super(NCE, self).__init__()
        self.U_x = nn.Sequential(
          nn.Linear(2, 20),
          nn.Tanh(),
          nn.Linear(20, 20),
          nn.Tanh(),
          nn.Linear(20, 20),
          nn.Tanh(),
          nn.Linear(20, 1),
        )
        self.beta = 0.05 
        self.muller_energy = compute_Muller_potential

    def forward(self, x):
        #return -self.beta*self.U_x(x) 
        return -self.beta*self.U_x(x)
        
    def ln_p_m(self, x): 
        #return -self.beta*self.U_x(x) 
        return -self.beta*self.U_x(x) #-self.muller_energy(self.beta, x) 
    
    def ln_p_n(self, noise_samples, noise_beta):
        return -self.muller_energy(noise_beta, noise_samples)
    
    def G_x_theta(self, x, noise_beta):
        return self.ln_p_m(x) - self.ln_p_n(x, noise_beta)

    def h_x_theta(self, x, noise_beta):
        return torch.sigmoid(self.G_x_theta(x, noise_beta))
    
    def loss(self, X_true, Y_true, noise_beta=0.05):
        #Y_true is a huge misnomer. Y_true is X_noise.
        T = X_true.size()[0] + Y_true.size()[0]
        J_T_vec = torch.log(self.h_x_theta(X_true, noise_beta)) + torch.log(1 - self.h_x_theta(Y_true, noise_beta)).to(device)
        return -(1/(2*T))*torch.sum(J_T_vec)

### Load Data and Create Basin1 and Basin2 Dataset

In [111]:
#bounds of the total area, but this is kind of meaningless
x1_min, x1_max = -1.5, 1
x2_min, x2_max = -0.5, 2.0

Abounds = [[-1.5, 0], [0.55, 2]] #basin 1 bounds. So -1.5 < x < 0, 0.55 < y < 2.
Bbounds = [[-0.8, 1], [-0.5, 0.8]] #basin 2 bounds. So -0.8 < x < 1, -0.5 < y < 0.8.

d = os.path.abspath('')
beta = 0.05

with open('Asamples_beta_{:.3f}.pkl'.format(beta), 'rb') as file_handle:
    dataA = pickle.load(file_handle)

xsamplesA = dataA['x_record']
betasA = dataA['beta_lst']

with open('Bsamples_beta_{:.3f}.pkl'.format(beta), 'rb') as file_handle:
    dataB = pickle.load(file_handle)

xsamplesB = dataA['x_record']
betasB = dataA['beta_lst']

betas = betasA

data_dictionaryA, data_dictionaryB = dict(), dict()

counter = 0
for beta in betasA:
    data_dictionaryA[float(beta)] = xsamplesA[:, counter, :]
    data_dictionaryB[float(beta)] = xsamplesB[:, counter, :]
    counter = counter + 1

List of defined betas: [0.0010, 0.0064, 0.0119, 0.0173, 0.0228, 0.0282, 0.0337, 0.0391, 0.0446, 0.0500].

### Train Model on Region A

In [112]:
device = torch.device("cpu")
modelA = NCE().to(device)
optimizerA = optim.Adam(modelA.parameters(), lr=0.5*1e-3)

def train_helper(model, optimizer, noise_samples, true_samples, btrue, bnoise):
    model.train()
    train_loss = 0

    optimizer.zero_grad()
    loss = model.loss(true_samples, noise_samples, bnoise)
    loss.backward()

    train_loss += loss.item()
    optimizer.step()
   

def train_main(model, optimizer, betas, data_dictionary, btrue, iters=10):
    true_samples = torch.tensor(data_dictionary[btrue]).to(device)
    for epoch in range(0, iters, 1):
        for beta in betas:
            noise_beta = float(beta)
            batchsize= 150 #150
            if noise_beta < 0.02: 
                print("Iter, Beta =", noise_beta, epoch)
                noise_samples = torch.tensor(data_dictionary[noise_beta]).to(device)
            
                for index in range(0, 200, 1):
                    train_helper(model, optimizer, noise_samples[index*150:(index+1)*150], 
                        true_samples[index*batchsize:(index+1)*batchsize], btrue, noise_beta)

btrue = 0.05
train_main(modelA, optimizerA, betas, data_dictionaryA, 0.05)

Iter, Beta = 0.001 0
Iter, Beta = 0.0064444444444444445 0
Iter, Beta = 0.01188888888888889 0
Iter, Beta = 0.017333333333333333 0
Iter, Beta = 0.001 1
Iter, Beta = 0.0064444444444444445 1
Iter, Beta = 0.01188888888888889 1
Iter, Beta = 0.017333333333333333 1
Iter, Beta = 0.001 2
Iter, Beta = 0.0064444444444444445 2
Iter, Beta = 0.01188888888888889 2
Iter, Beta = 0.017333333333333333 2
Iter, Beta = 0.001 3
Iter, Beta = 0.0064444444444444445 3
Iter, Beta = 0.01188888888888889 3
Iter, Beta = 0.017333333333333333 3
Iter, Beta = 0.001 4
Iter, Beta = 0.0064444444444444445 4
Iter, Beta = 0.01188888888888889 4
Iter, Beta = 0.017333333333333333 4
Iter, Beta = 0.001 5
Iter, Beta = 0.0064444444444444445 5
Iter, Beta = 0.01188888888888889 5
Iter, Beta = 0.017333333333333333 5
Iter, Beta = 0.001 6
Iter, Beta = 0.0064444444444444445 6
Iter, Beta = 0.01188888888888889 6
Iter, Beta = 0.017333333333333333 6
Iter, Beta = 0.001 7
Iter, Beta = 0.0064444444444444445 7
Iter, Beta = 0.01188888888888889 7
Iter

### Train Model on Region B

In [113]:
device = torch.device("cpu")
modelB = NCE().to(device)
optimizerB = optim.Adam(modelB.parameters(), lr=0.5*1e-3)
btrue = 0.05
train_main(modelB, optimizerB, betas, data_dictionaryB, 0.05)

Iter, Beta = 0.001 0
Iter, Beta = 0.0064444444444444445 0
Iter, Beta = 0.01188888888888889 0
Iter, Beta = 0.017333333333333333 0
Iter, Beta = 0.001 1
Iter, Beta = 0.0064444444444444445 1
Iter, Beta = 0.01188888888888889 1
Iter, Beta = 0.017333333333333333 1
Iter, Beta = 0.001 2
Iter, Beta = 0.0064444444444444445 2
Iter, Beta = 0.01188888888888889 2
Iter, Beta = 0.017333333333333333 2
Iter, Beta = 0.001 3
Iter, Beta = 0.0064444444444444445 3
Iter, Beta = 0.01188888888888889 3
Iter, Beta = 0.017333333333333333 3
Iter, Beta = 0.001 4
Iter, Beta = 0.0064444444444444445 4
Iter, Beta = 0.01188888888888889 4
Iter, Beta = 0.017333333333333333 4
Iter, Beta = 0.001 5
Iter, Beta = 0.0064444444444444445 5
Iter, Beta = 0.01188888888888889 5
Iter, Beta = 0.017333333333333333 5
Iter, Beta = 0.001 6
Iter, Beta = 0.0064444444444444445 6
Iter, Beta = 0.01188888888888889 6
Iter, Beta = 0.017333333333333333 6
Iter, Beta = 0.001 7
Iter, Beta = 0.0064444444444444445 7
Iter, Beta = 0.01188888888888889 7
Iter

### Display The Samples

In [115]:
plt.scatter(samplesA[0], samplesA[1])

TypeError: scatter() missing 1 required positional argument: 'y'

### DeepFEP

The free energy difference should be: -28.46.

In [114]:
def E_A(samples):
    return (modelA(torch.tensor(samples).reshape(len(samples), 2).to(device))).cpu().detach().numpy()

def E_B(samples):
    return (modelB(torch.tensor(samples).reshape(len(samples), 2).to(device))).cpu().detach().numpy()

samplesA = torch.tensor(data_dictionaryA[0.05])[0:10]
samplesB = torch.tensor(data_dictionaryB[0.05])[0:10]

F_A = (-1/beta)*np.log(
    np.sum(
        np.exp( E_A(samplesA) - np.array(compute_Muller_potential(0.05, samplesA)) )
        )
    )

F_B = (-1/beta)*np.log(
    np.sum(
        np.exp( E_B(samplesB) - np.array(compute_Muller_potential(0.05, samplesB)) )
        )
    )

print(F_A)
print(F_B)
print(F_A - F_B)

tensor(-282.3481)
tensor(-279.8099)
tensor(-2.5383)


<ipython-input-114-56cf5d4a23c9>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return (modelA(torch.tensor(samples).reshape(len(samples), 2).to(device))).cpu().detach().numpy()
<ipython-input-114-56cf5d4a23c9>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return (modelB(torch.tensor(samples).reshape(len(samples), 2).to(device))).cpu().detach().numpy()
